In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3.2")

C:\Users\dhananjayl\AppData\Local\Temp\ipykernel_27772\2407945860.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "llama3.2")


In [2]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
import os

In [1]:
oracleUri = f"oracle+oracledb://{"HR"}:{"HR"}@(DESCRIPTION = (ADDRESS = (PROTOCOL = TCP)(HOST = {"192.168.59.11"})(PORT = {1521}))(CONNECT_DATA = (SERVICE_NAME = {"XEPDB1"})))"
db = SQLDatabase.from_uri(oracleUri)

NameError: name 'SQLDatabase' is not defined

In [5]:
chain = create_sql_query_chain(llm = llm, db = db)

In [ ]:
"""
                    Given an input question, create a syntactically correct Oracle query to run.\n
                    Here is the relevant table info: {table_info}\n
                    Pay attention to use RUMNUM clause to obtain top 5 rows number of rows from the query.Unless the user specifies in the question a {top_k} number of examples to obtain using rownum clause.
                    Never query for all columns from a table.\n
                    The sql should not have ``` in the beginning and end.\n you should avoid using the word sql in your SQL query output.
                    The output should not have \\n in the sql query.\n
                    Also generate full queries for the given user question.\nDont include ``` and \\n in the output.
                    DO NOT make any DML statements (INSERT,UPDATE, DELETE, DROP etc). to the database.\n
                    If the question does not seem related to the database, just return 'I  don't know' as the answer.
                    If you don't know answer, mention that you don't know the answer.\n
                    \nPay attention to use CURDATE() function to get the current date, if the question involves "today".
                    \nUse the following format: 
                    SQLQuery- SQL Query to run
                    Assumptions-Atleast 5 Assumptions you made to answer the question as bullet points\n
                    Below are a number of examples of questions and their corresponding SQL queries."""

In [15]:
database_description = """
"You are a expert in Oracle who can convert english questions to corresponding Oracle Query."
"This is a Oracle database, so please ensure that all queries are compatible with oracle syntax.
{
    "table_name": "EMPLOYEES",
    "comments": "Employees table with 107 rows, referencing departments, jobs, and job history. Contains a self-reference.",
    "columns": [
        {"name": "EMPLOYEE_ID", "type": "NUMBER", "comments": "Primary key."},
        {"name": "DEPARTMENT_ID", "type": "NUMBER", "comments": "Foreign key to DEPARTMENTS.DEPARTMENT_ID."},
        {"name": "MANAGER_ID", "type": "NUMBER", "comments": "Foreign key to EMPLOYEES.EMPLOYEE_ID (self-referential)."},
        {"name": "COMMISSION_PCT", "type": "NUMBER", "comments": "Commission percentage (only for sales employees)."},
        {"name": "SALARY", "type": "NUMBER", "comments": "Must be greater than zero."},
        {"name": "JOB_ID", "type": "VARCHAR2", "comments": "Foreign key to JOBS.JOB_ID. Not null."},
        {"name": "HIRE_DATE", "type": "DATE", "comments": "Employee start date. Not null."},
        {"name": "PHONE_NUMBER", "type": "VARCHAR2", "comments": "Includes country and area code."},
        {"name": "EMAIL", "type": "VARCHAR2", "comments": "Employee email."},
        {"name": "LAST_NAME", "type": "VARCHAR2", "comments": "Not null."},
        {"name": "FIRST_NAME", "type": "VARCHAR2", "comments": "Not null."}
    ]
}
"SQLQuery: Please write your query below following above table schema."
"""

In [27]:
# database_description = (
#     "You are a expert in Oracle who can convert english questions to corresponding Oracle Query."
#     "This is a Oracle database, so please ensure that all queries are compatible with oracle syntax.\n\n"
#     "SQLQuery: Please write your query below following this format."
# )

response = chain.invoke({"question": database_description + " total number of employees ?"})

c:\Users\dhananjayl\AppData\Local\miniconda3\envs\llama\Lib\site-packages\langchain_community\utilities\sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "departments, employees", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Users\dhananjayl\AppData\Local\miniconda3\envs\llama\Lib\site-packages\langchain_community\utilities\sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "departments, employees", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables


In [31]:
import re
def extract_sql_query(response):
    # Define the regular expression pattern to match the SQL query
    pattern = re.compile(r'SQLQuery:\s*(.*);')
    # pattern = re.compile(r'SQLQuery:\s*```(.*?)```', re.DOTALL)

    print(f"pattern:{pattern}")
    # Search for the pattern in the response
    match = pattern.search(response)
    print(f"match:{match}")
    if match:
        # Extract and return the matched SQL query
        return match.group(1).strip()
    else:
        return None
sql_query = extract_sql_query(response)
print(sql_query)

pattern:re.compile('SQLQuery:\\s*(.*);')
match:<re.Match object; span=(47, 90), match='SQLQuery:\nSELECT COUNT(*) FROM "EMPLOYEES";'>
SELECT COUNT(*) FROM "EMPLOYEES"


In [32]:
result = db.run(sql_query)
print(result)

[(107,)]


In [33]:
print(response)

Answer: The total number of employees is 107.

SQLQuery:
SELECT COUNT(*) FROM "EMPLOYEES"; 

Assumptions:

* The query only counts rows where EMPLOYEE_ID, DEPARTMENT_ID, and JOB_ID are not NULL to ensure that there is no self-referential issue.
* The use of TRUNC(SYSDATE) function was omitted since the HIRE_DATE column already represents the date when each employee joined the company.
* The query does not require any additional joins or subqueries because it can directly count the number of rows in the "EMPLOYEES" table.


In [21]:
db.run('''SELECT "FIRST_NAME"
FROM "EMPLOYEES"
WHERE "FIRST_NAME" = 'Lex'
ORDER BY "FIRST_NAME" ASC
FETCH FIRST 1 ROWS ONLY''')

"[('Lex',)]"

In [ ]:
from db_schema import fetch_db_schema

fetch_db_schema(eng)